# CNN（畳み込みニューラルネットワーク）の基礎

このノートブックでは、画像分類の数理的基礎を学びます。

## 目次
1. 画像データの構造
2. CNNの全体像
3. 手書き数字分類（MNIST）

## 1. 画像データの構造

画像データは**ピクセル**（画素）の集合体です。

- **グレースケール画像**: 2次元配列 (H, W)、各ピクセルは0〜255の濃淡値
- **カラー画像**: 3次元配列 (H, W, 3)、RGB（Red/Green/Blue）の3チャンネル

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# グレースケール画像の例（8x8ピクセル）
gray_image = np.array([
    [0, 0, 50, 100, 100, 50, 0, 0],
    [0, 50, 150, 200, 200, 150, 50, 0],
    [50, 150, 255, 255, 255, 255, 150, 50],
    [100, 200, 255, 200, 200, 255, 200, 100],
    [100, 200, 255, 200, 200, 255, 200, 100],
    [50, 150, 255, 255, 255, 255, 150, 50],
    [0, 50, 150, 200, 200, 150, 50, 0],
    [0, 0, 50, 100, 100, 50, 0, 0]
], dtype=np.uint8)

print(f"画像の形状: {gray_image.shape}")
print(f"ピクセル値の範囲: {gray_image.min()} 〜 {gray_image.max()}")

plt.figure(figsize=(6, 6))
plt.imshow(gray_image, cmap='gray', vmin=0, vmax=255)
plt.colorbar(label='ピクセル値')
plt.title('グレースケール画像の例')
plt.show()

## 2. CNNの全体像

CNNは以下の処理を繰り返して画像を分類します：

```
入力画像 → [畳み込み → 特徴マップ → プーリング（圧縮）] × N → 全結合層 → 分類結果
```

### 主要な処理
1. **畳み込み（Convolution）**: フィルタを使って特徴を抽出
2. **プーリング（Pooling）**: 特徴マップのサイズを圧縮
3. **全結合層（Fully Connected）**: 最終的な分類を行う

## 3. 手書き数字分類（MNIST）

MNISTデータセット：28x28ピクセルのグレースケール手書き数字（0〜9）

In [ ]:
# MNISTデータセットの読み込み
import torch
from torchvision import datasets, transforms

# データの前処理
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  # MNIST用の正規化
])

# データセットのダウンロード
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

print(f"訓練データ: {len(train_dataset)} 枚")
print(f"テストデータ: {len(test_dataset)} 枚")

In [ ]:
# サンプル画像の表示
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
for i, ax in enumerate(axes.flat):
    image, label = train_dataset[i]
    ax.imshow(image.squeeze(), cmap='gray')
    ax.set_title(f'ラベル: {label}')
    ax.axis('off')
plt.tight_layout()
plt.show()

## 次のステップ

次のノートブックでは、畳み込み演算の数理について詳しく学びます。